# Exploring Hacker News Posts

[Hacker News](https://news.ycombinator.com/) is a popular site in technology and startup circles, where user-submitted stories (known as "posts") are voted and commented upon, similar to reddit. In this project we will focus on analyzing posts whose titles begin with either __Ask HN__ or __Show HN__.  

Users submit __Ask HN__ posts to ask the Hacker News community a specific question, for example: _Ask HN: How to improve my personal website?_ Likewise, users submit __Show HN__ posts to show the Hacker News community a project, product, or just generally something interesting. For example: _Show HN: Wio Link  ESP8266 Based Web of Things Hardware Development Platform'_.

We'll specifically compare these two types of posts to determine the following:  
- __Do Ask HN or Show HN receive more comments on average?__  
- __Do posts created at a certain time receive more comments on average?__

---
## Dataset

We will be working with a [dataset](https://www.kaggle.com/hacker-news/hacker-news-posts) reduced from almost 300,000 rows to approximately 20,000 rows by removing all submissions that did not receive any comments, and then randomly sampling from the remaining submissions.

Below are descriptions of the columns:
- <code>id</code> - the unique identifier from Hacker News for the post
- <code>title</code> - the title of the post
- <code>url</code> - the URL that the posts links to, if the post has a URL
- <code>num_points</code> - the number of points the post acquired, calculated as the total number of upvotes minus the total number of downvotes
- <code>num_comments</code> - the number of comments on the post
- <code>author</code> - the username of the person who submitted the post
- <code>created_at</code> - the date and time of the post's submission

In [1]:
# Read in the data
import csv

opened_file = open("hacker_news.csv")
hn = list(csv.reader(opened_file))

# Removing header
hn_header = hn[0]
hn = hn[1:]

print(hn_header)
print(hn[:5])

['id', 'title', 'url', 'num_points', 'num_comments', 'author', 'created_at']
[['12224879', 'Interactive Dynamic Video', 'http://www.interactivedynamicvideo.com/', '386', '52', 'ne0phyte', '8/4/2016 11:52'], ['10975351', 'How to Use Open Source and Shut the Fuck Up at the Same Time', 'http://hueniverse.com/2016/01/26/how-to-use-open-source-and-shut-the-fuck-up-at-the-same-time/', '39', '10', 'josep2', '1/26/2016 19:30'], ['11964716', "Florida DJs May Face Felony for April Fools' Water Joke", 'http://www.thewire.com/entertainment/2013/04/florida-djs-april-fools-water-joke/63798/', '2', '1', 'vezycash', '6/23/2016 22:20'], ['11919867', 'Technology ventures: From Idea to Enterprise', 'https://www.amazon.com/Technology-Ventures-Enterprise-Thomas-Byers/dp/0073523429', '3', '1', 'hswarna', '6/17/2016 0:01'], ['10301696', 'Note by Note: The Making of Steinway L1037 (2007)', 'http://www.nytimes.com/2007/11/07/movies/07stein.html?_r=0', '8', '2', 'walterbell', '9/30/2015 4:12']]


We can see above that the data set contains the title of the posts, the number of comments for each post, and the date the post was created. Let's start by exploring the number of comments for each type of post.

---
## Separating Ask HN and Show HN posts

Since we're only concerned with post titles beginning with __Ask HN__ or __Show HN__, we'll create new lists of lists containing just the data for those titles.

In [2]:
# Identify posts which start with 'Ask HN' or 'Show HN' and save them in separate lists

ask_posts = []
show_posts = []
other_posts = []

for row in hn:
    title = row[1]
    if title.lower().startswith('ask hn'):
        ask_posts.append(row)
    elif title.lower().startswith('show hn'):
        show_posts.append(row)
    else:
        other_posts.append(row)
        
print('Number of ask posts: ', len(ask_posts))
print('Number of show posts: ', len(show_posts))
print('Number of other posts: ', len(other_posts))

Number of ask posts:  1744
Number of show posts:  1162
Number of other posts:  17194


## Calculating the average number of comments for ask and show posts

Let's determine if ask posts or show posts receive more comments. To do that, we will calculate the average number of comments each type of post receives.

In [3]:
# Calculate average number of comments for ask hn posts
total_ask_comments = 0
for row in ask_posts:
    comments = int(row[4])
    total_ask_comments += comments
    
avg_ask_comments = total_ask_comments / len(ask_posts)
print('Average number of comments of Ask HN posts: ', round(avg_ask_comments, 2))

Average number of comments of Ask HN posts:  14.04


In [4]:
# Calculate average number of comments for show hn posts
total_show_comments = 0
for row in show_posts:
    comments = int(row[4])
    total_show_comments += comments
    
avg_show_comments = total_show_comments / len(show_posts)
print('Average number of comments of Show HN posts: ', round(avg_show_comments,2))

Average number of comments of Show HN posts:  10.32


On average __ask posts receive 14 comments__ per post, while __show post have 10 comments__. Since ask posts receive more comments the rest of the analysis will be focused on this group of posts.

## Finding the amount of ask posts and comments by hour created

We are going to explore if posts created at a certain time are attracting more comments. First, we'll find the amount of ask posts created during each hour of day, along with the number of comments those posts received. Then, we'll calculate the average amount of comments ask posts created at each hour of the day receive.

In [5]:
# Calculate the amount of ask posts created during each hour of day and the number of comments received.
import datetime as dt

result_list = []
for post in ask_posts:
    result_list.append(
        [post[6], int(post[4])] # creation date, number of comments
    ) 
    
counts_by_hour = {}
comments_by_hour = {}
date_format = "%m/%d/%Y %H:%M"

for row in result_list:
    date = row[0]
    comments = row[1]
    time = dt.datetime.strptime(date, date_format).strftime("%H")
    if time not in counts_by_hour:
        counts_by_hour[time] = 1
        comments_by_hour[time] = comments
    else: 
        counts_by_hour[time] += 1
        comments_by_hour[time] += comments
        
print(comments_by_hour)

{'09': 251, '13': 1253, '10': 793, '14': 1416, '16': 1814, '23': 543, '12': 687, '17': 1146, '15': 4477, '21': 1745, '20': 1722, '02': 1381, '18': 1439, '03': 421, '05': 464, '19': 1188, '01': 683, '22': 479, '08': 492, '04': 337, '00': 447, '06': 397, '07': 267, '11': 641}


## Calculating average number of comments per post created during each hour of the day

To calculate the average number of comments for posts created during each hour of the day we will use the dictionaries created in the previous step:
- <code>counts_by_hour</code> - contains the number of ask posts created during each hour of the day.
- <code>comments_by_hour</code> - contains the corresponding number of comments ask posts created at each hour received

In [6]:
# Calculate the average amount of comments `Ask HN` posts created at each hour of the day receive.
avg_by_hour = []

for key in comments_by_hour:
    avg_by_hour.append([key, comments_by_hour[key] / counts_by_hour[key]])

print(avg_by_hour)

[['09', 5.5777777777777775], ['13', 14.741176470588234], ['10', 13.440677966101696], ['14', 13.233644859813085], ['16', 16.796296296296298], ['23', 7.985294117647059], ['12', 9.41095890410959], ['17', 11.46], ['15', 38.5948275862069], ['21', 16.009174311926607], ['20', 21.525], ['02', 23.810344827586206], ['18', 13.20183486238532], ['03', 7.796296296296297], ['05', 10.08695652173913], ['19', 10.8], ['01', 11.383333333333333], ['22', 6.746478873239437], ['08', 10.25], ['04', 7.170212765957447], ['00', 8.127272727272727], ['06', 9.022727272727273], ['07', 7.852941176470588], ['11', 11.051724137931034]]


## Sorting the results to identify hours with the highest values

In order to better read the results, we will sort the dictionary in descending order by the average number of comments per post.

In [7]:
swap_avg_by_hour = []
for row in avg_by_hour:
    swap_avg_by_hour.append([row[1], row[0]])

print(swap_avg_by_hour)

sorted_swap = sorted(swap_avg_by_hour, reverse = True)

print(sorted_swap)

[[5.5777777777777775, '09'], [14.741176470588234, '13'], [13.440677966101696, '10'], [13.233644859813085, '14'], [16.796296296296298, '16'], [7.985294117647059, '23'], [9.41095890410959, '12'], [11.46, '17'], [38.5948275862069, '15'], [16.009174311926607, '21'], [21.525, '20'], [23.810344827586206, '02'], [13.20183486238532, '18'], [7.796296296296297, '03'], [10.08695652173913, '05'], [10.8, '19'], [11.383333333333333, '01'], [6.746478873239437, '22'], [10.25, '08'], [7.170212765957447, '04'], [8.127272727272727, '00'], [9.022727272727273, '06'], [7.852941176470588, '07'], [11.051724137931034, '11']]
[[38.5948275862069, '15'], [23.810344827586206, '02'], [21.525, '20'], [16.796296296296298, '16'], [16.009174311926607, '21'], [14.741176470588234, '13'], [13.440677966101696, '10'], [13.233644859813085, '14'], [13.20183486238532, '18'], [11.46, '17'], [11.383333333333333, '01'], [11.051724137931034, '11'], [10.8, '19'], [10.25, '08'], [10.08695652173913, '05'], [9.41095890410959, '12'], [

In [8]:
# Print the top 5 hours with the highest average comments.
print("Top 5 Hours for Ask Posts Comments")
for avg, hr in sorted_swap[:5]:
    print(
        "{}: {:.2f} average comments per post".format(
            dt.datetime.strptime(hr, "%H").strftime("%H:%M"), avg)
    )

Top 5 Hours for Ask Posts Comments
15:00: 38.59 average comments per post
02:00: 23.81 average comments per post
20:00: 21.52 average comments per post
16:00: 16.80 average comments per post
21:00: 16.01 average comments per post


The hour that receives the most comments per post on average is __15:00__, with an average of __38.59 comments__ per post. According to the data set [documentation](https://www.kaggle.com/hacker-news/hacker-news-posts/home), the timezone used is Eastern Time in the US.

---
## Conclusion

Refering back to the goal of this study, let's summarize the conclusions. We analyzed ask posts and show posts to determine which type of post and time receive the most comments on average. 

__Post type__: based on our analysis, to maximize the amount of comments a post receives, we'd recommend the post be categorized as __Ask HN post__:
- __Average number of comments of Ask HN posts:  14.04__
- Average number of comments of Show HN posts:  10.32

__Post timing__: the time of day of posting appears to have significant impact on the number of comments that you will attract. Based on an analysis of the Ask HN posts, the top hours (in est time) are:
- __15:00: 38.59 average comments per post__
- 02:00: 23.81 average comments per post
- 20:00: 21.52 average comments per post

It should be noted that the data set we analyzed excluded posts without any comments. Given that, it's more accurate to say that of the posts that received comments, ask posts received more comments on average and ask posts created between 15:00 and 16:00 received the most comments on average.